# Scraper TOUT CineAtlas avec des fonctions

In [1]:
import re
import json
import requests
from ast import literal_eval

url = 'https://www.cineatlas.com/'

html_data = requests.get(url).text
movieData = re.findall(r'movieData = ({.*?}), movieDataByReleaseDate', html_data, flags=re.DOTALL)[0]
movieData = re.sub(r'\s*/\*.*?\*/\s*', '', movieData)   # remove comments
movieData = literal_eval(movieData) # in movieData you have now the information about the current movies

### Formater le JSON pour plus de lisibilité

In [2]:
jfile = json.dumps(movieData, indent=4)

In [3]:
data = json.loads(jfile)

## Scraper les dates

Le fichier JSON est défini date par date. Il faut d'abord isoler ces données pour les réutiliser avec les fonctions

In [4]:
dates = [ ]
for k in data:
    dates.append(k)

# ---------------    Fonctions   -------------------------

## Liens :

In [5]:
def scrape_liens(date):
    mLiens = []
    for i in data.get(date):
        mLiens.append("https://www.cineatlas.com/movie/"+str(i['url']))
    return mLiens

## Noms :

In [6]:
def scrape_noms(date):
    mNoms = []
    for i in data.get(date):
        mNoms.append((i['title']))
    return mNoms

## Images : 

In [7]:
def scrape_images(date):
    mImages = []
    for i in data.get(date):
        mImages.append((i['image-portrait']))
    return mImages
    

## Heures :

In [8]:
def scrape_heures(date):
    mHeures = []
    heures = []
    
    for i in data.get(date):
        h = []
        for j in i['times']:
            h.append(j['time'])
        heures.append(h)
    
    for i in heures:
        s = ""
        for j in i:
            s =s+str(j)+" "
        mHeures.append(s)     
        
    return mHeures    

## Descriptions :

In [9]:
from time import time
from time import sleep
from random import randint

In [10]:
from bs4 import BeautifulSoup
from requests import get

In [11]:
def scrape_descriptions(links):
    
    mDescs = []
    
    start_time = time()
    requests = 0
    
   
    for i in links:
        response = get(i)
    
        timer = randint(8,15)
        #pause de loop
        print(timer)
        sleep(timer)
    
        #renvoyer un warning pour les non 'status code : 200'
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
        #parser en BS
        page_html = BeautifulSoup(response.text,'html.parser')  
    
        events = page_html.find('div',class_='mobile-hide movie-details-container')
    
        if events is None:
            mDescs.append("NO DESCRIPTION")
            continue
    
        event = events.find('div',class_='movie-details')   
    
        
        #dans le trou du cul du code (aka 4eme <p> du <div> cible )
    
        mDescs.append(event.findAll('p',recursive=False)[3].text)
        
    print("** Done **") 
    
    return mDescs
    
     
    
    
    

# ------------------- TESTER TOUT LE BORDEL   ------------------

In [12]:
dates

['2019-08-11', '2019-08-12', '2019-08-13']

In [13]:
dates[0]

'2019-08-11'

In [14]:
super_prog =[]
for date in dates:
    
    a = scrape_liens(date)
    b = scrape_noms(date)
    c = scrape_images(date)
    d = scrape_heures(date)
    e = scrape_descriptions(a)
    
    prog = []
    for i,j,k,l,m in zip(b,d,c,a,e):
        #Ajouter le lieu
        t = (i,date,j,k,l,m,"Cine Atlas")
        prog.append(t)
        
    super_prog = super_prog + prog
    
    print(" \n \n ******* Palier "+str(date)+"******* \n \n")

print(" #############\n \n ######## BIG DONE ######## \n \n")
    
    

13
11
10
9
10
9
11
14
11
** Done **
 
 
 ******* Palier 2019-08-11******* 
 

14
12
9
14
11
14
** Done **
 
 
 ******* Palier 2019-08-12******* 
 

10
15
9
15
12
15
13
** Done **
 
 
 ******* Palier 2019-08-13******* 
 

 #############
 
 ######## BIG DONE ######## 
 



In [20]:
super_prog

[('FAST &amp; FURIOUS : HOBBS &amp; SHAW',
  '2019-08-11',
  '12:30pm 12:30pm 3:30pm 6:25pm 9:15pm ',
  'https://d10u9ygjms7run.cloudfront.net/dd2qd1xaf4pceqxvb41s1xpzs0/1562603443098_891497ecc8b16b3a662ad8b036820ed1_500x735.jpg',
  'https://www.cineatlas.com/movie/fast--furious--hobbs--shaw',
  'Depuis que Hobbs, fidèle agent de sécurité au service diplomatique des Etats-Unis, combatif mais droit, et Shaw, un homme sans foi ni loi, ancien membre de l’élite militaire britannique, se sont affrontés en 2015 dans Fast & Furious 7 ; les deux hommes font tout ce qu’ils peuvent pour se nuire l’un à l’autre.Mais lorsque Brixton, un anarchiste génétiquement modifié, met la main sur une arme de destruction massive après avoir battu le meilleur agent du MI6 qui se trouve être la sœur de Shaw. Les deux ennemis de longue date vont devoir alors faire équipe pour faire tomber le seul adversaire capable de les anéantir.',
  'Cine Atlas'),
 ('TOY STORY 4',
  '2019-08-11',
  '10:30am ',
  'https://d10u

## Convertir en objet pandas

In [21]:
import pandas as pd

In [22]:
df = pd.DataFrame(super_prog,columns=['noms','date','heure','image','lien','description','lieu'])

In [23]:
df

,noms,date,heure,image,lien,description,lieu
0,FAST &amp; FURIOUS : HOBBS &amp; SHAW,2019-08-11,12:30pm 12:30pm 3:30pm 6:25pm 9:15pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/fast--furious-...,"Depuis que Hobbs, fidèle agent de sécurité au ...",Cine Atlas
1,TOY STORY 4,2019-08-11,10:30am,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/ST00000406-toy...,NO DESCRIPTION,Cine Atlas
2,Comme des bêtes 2,2019-08-11,10:30am 12:30pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/comme-des-btes-2,"La suite du film d'animation ""Comme des bêtes""...",Cine Atlas
3,UGLYDOLLS,2019-08-11,10:30am,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/uglydolls,"À Uglyville, on aime tout ce qui est bizarre, ...",Cine Atlas
4,Midsommar,2019-08-11,7:10pm 9:55pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/midsommar,Dani and Christian are a young American couple...,Cine Atlas
5,ANNABELLE LA MAISON DU MAL,2019-08-11,7:30pm 9:40pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/annabelle-la-m...,Déterminés à mettre Annabelle hors d'état de n...,Cine Atlas
6,LE ROI LION,2019-08-11,10:30am 1:00pm 3:10pm 5:35pm 7:55pm 10:20pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/ST00000442-le-...,NO DESCRIPTION,Cine Atlas
7,LE COUP DU SIECLE / THE HUSTLE,2019-08-11,3:10pm 5:10pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/le-coup-du-sie...,"In the hilarious new comedy THE HUSTLE, Anne H...",Cine Atlas
8,SPIDER-MAN:FAR FROM HOME,2019-08-11,2:20pm 4:55pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/spider-manfar-...,L'araignée sympa du quartier décide de rejoind...,Cine Atlas
9,FAST &amp; FURIOUS : HOBBS &amp; SHAW,2019-08-12,3:30pm 6:25pm 9:15pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/fast--furious-...,"Depuis que Hobbs, fidèle agent de sécurité au ...",Cine Atlas
